In [ ]:
from cyschoolhousesuite import *
from selenium.webdriver.support.ui import Select
import datetime
import xlwings as xw
import numpy as np
import os

In [ ]:
path_to_xlsx = r'Z:\ChiPrivate\Chicago Data and Evaluation\SY18\SY18 Students to Upload to cyschoolhouse.xlsx'
temp_folder = r'C:\Users\City_Year\GitHub\cy-automation-library\cyautomation\cyschoolhouse\temp\\'

def fill_template(df):
    """Imports input data from xlsx
    """
    df = df.loc[df["STATUS"].isnull() & ~df["*REQ* School"].isnull()].copy()
    df.loc[:,'Date of Birth'] = df['Date of Birth'].dt.strftime('%m/%d/%Y')
    df['Date of Birth'].loc[df['Date of Birth']=="NaT"] = np.nan
    
    cols = ['*REQ* School',
            '*REQ* Student Id',
            '*REQ* First Name',
            '*REQ* Last Name',
            '*REQ* Grade',
            'Date of Birth',
            'Gender',
            'Ethnicity',
            'Disability Flag',
            'ELL',
            'Entry Date',
            'Email Address',
            'Address',
            'City',
            'State',
            'Zip',
            'Phone',
            '*REQ* Type',
            'Local Student ID']

    null_df = pd.DataFrame(columns=cols)

    df = pd.concat([null_df, df])[cols]
    
    df["Entry Date"] = "7/1/2017"
    df["*REQ* Type"] = "Student"
    df["Local Student ID"] = df["*REQ* Student Id"]
    df["*REQ* Student Id"] = range(0, len(df))
    df["ELL"].replace([0.0, 1.0],["FALSE", "TRUE"], inplace=True)
    
    return df

def write_csv(params, school_name):
    df_csv = params.loc[params["*REQ* School"]==school_name].copy()
    df_csv.drop(["*REQ* School"], axis=1, inplace=True)
    df_csv.to_csv("temp/" + school_name + ".csv", index=False, date_format='%m/%d/%Y')

def nav_to_intervention_setup(driver):
    # Maybe not the best url:
    driver.get('https://c.na24.visual.force.com/apex/IM_InterventionsSetup?sfdc.tabName=01r1a000000QQ87')
    
def input_school_name(driver, school_name):
    dropdown = Select(driver.find_element_by_id("j_id0:j_id49:schools"))
    dropdown.select_by_visible_text(school_name)
    sleep(.5)
    driver.find_element_by_css_selector('input.drk_blue_btn').click()
    
def load_from_csv(driver, temp_folder, school_name):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/table/tbody/tr/td/table/tbody/tr/td[2]/div/form/div[1]/div[1]/input[1]')))
    driver.find_element_by_css_selector('input.black_btn').click()

    driver.switch_to.window(driver.window_handles[1])
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j_id0:j_id42:objectTypeSelect")))
    dropdown = Select(driver.find_element_by_id("j_id0:j_id42:objectTypeSelect"))
    dropdown.select_by_visible_text('Student Holding Object Template')
    
    driver.find_element_by_xpath('//*[@id="selectedFile"]').send_keys(temp_folder + school_name + ".csv")
    driver.find_element_by_xpath(".//input[contains(@type, 'button') and contains(@value, 'Upload File')]").click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="output"]')))
    
    sleep(3)
    
    # Fails to click this button at the right moment without a sleep() before it
    driver.find_element_by_xpath(".//input[contains(@id, 'startBatchButton') and contains(@value, 'Insert Data')]").click()
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//img[contains(@src, '/img/msg_icons/confirm24.png')]")))
    driver.find_element_by_xpath(".//button[contains(@class, 'ui-button ui-widget ui-state-default ui-corner-all ui-button-text-only') and contains(@type, 'button')]").click()
    
    driver.close()
    
def publish(driver):
    # navigate back to first tab
    driver.switch_to.window(driver.window_handles[0])
    # click button to publish
    driver.find_element_by_css_selector('input.red_btn').click()
    # Wait for Publishing progress box to appear and disappear
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, ".//div[contains(@id, 'fancybox-wrap')]")))
    WebDriverWait(driver, 120).until(EC.invisibility_of_element_located((By.XPATH, ".//div[contains(@id, 'fancybox-wrap')]")))
    
def run_intervention_setup():
    students_df = pd.read_excel(path_to_xlsx, converters={'*REQ* Grade':int})
    params = fill_template(df=students_df)
    driver = get_driver()
    open_cyschoolhouse18(driver)
    for school_name in params['*REQ* School'].unique():
        try:
            write_csv(params, school_name)
            nav_to_intervention_setup(driver)
            input_school_name(driver, school_name)
            load_from_csv(driver, temp_folder, school_name)
            publish(driver)
        except Exception as e:
            print(f"Failed: {school_name}: {e}")
            continue
        students_df["STATUS"].loc[(students_df['*REQ* School'] == school_name)
                                  & (students_df["STATUS"].isnull())] = "Uploaded " + datetime.datetime.today().strftime('%m/%d/%Y')
        try:
            os.remove("temp/" + school_name + ".csv")
        except Exception as e:
            print(f"Failed to remove local file: {school_name}: {e}")
    driver.quit()
    
    wb = xw.Book("templates/Students to Upload to cyschoolhouse Template.xlsx")
    sht = wb.sheets['Students to Upload']
    sht.range('A2:Z900').clear_contents()
    sht.range('A2').options(index=False, header=False).value = students_df

    try: 
        wb.save(path_to_xlsx)
    except Exception as e:
        print("Error: could not save .xlsx, update manually: {e}")
    wb.close()
    
if __name__ == '__main__':
    run_intervention_setup()